In [1]:
import numpy as np
from scipy.sparse import csc_array,csr_array,diags_array
from scipy.sparse import random_array
import matplotlib.pyplot as plt
from scipy.special import expit
from numpy import logaddexp
from tqdm.auto import tqdm
from scipy.optimize import minimize_scalar
from scipy.sparse.linalg import spsolve
import qdldl
from scipy.sparse import block_array,eye_array
import scipy
from sparse_dot_mkl import dot_product_mkl
import time
from util import PrettyLogger,get_step_size,maxnorm,norm2
from warnings import warn

In [2]:
class LogisticNLL:
    def __init__(self, y, w):
        """
        y: array-like of binary responses (0 or 1)
        w: array-like of corresponding weights
        """
        self.y = np.array(y, dtype=float)
        self.w = np.array(w, dtype=float)

    def f(self, z):
        """
        Computes the negative weighted logistic log-likelihood:
            sum_i w_i [ log(1 + exp(z_i)) - y_i * z_i ]
        """
        z = np.array(z, dtype=float)
        return np.sum(self.w * (logaddexp(0, z) - self.y * z))

    def d1f(self, z):
        """
        Computes the first derivative (gradient) w.r.t. z:
            w_i [ sigma(z_i) - y_i ]
        """
        z = np.array(z, dtype=float)
        sig_z = expit(z)
        return self.w * (sig_z - self.y)

    def d2f(self, z):
        """
        Computes the second derivative (Hessian diagonal) w.r.t. z:
            w_i * sigma(z_i) * (1 - sigma(z_i))
        also using expit(z) for numerical stability.
        """
        z = np.array(z, dtype=float)
        sig_z = expit(z)
        hess_diag = self.w * sig_z * (1.0 - sig_z)
        return hess_diag

    def __call__(self, z):
        """
        By defining __call__, this object can be used like a function:
            phi(z) = phi.f(z)
        """
        return self.f(z)

In [20]:
from scipy.stats import Uniform


m = 10000
n = 500
samp = Uniform(a = -0.5,b = 0.5).sample


A = random_array((m,n),density = 0.01,data_sampler = lambda size:samp(size))
x_true = np.random.uniform(-0.1,1,n)

z_true = A@x_true

w = 100*np.ones(m)
y = np.random.binomial(w.astype(int),expit(z_true))/w

Q = 1.*diags_array(np.ones(n))

rng = np.random.default_rng(10)

xx = rng.normal(size = n)
samp = Uniform(a = -0.5,b = 0.5).sample

C = random_array((200,n),
                 density = 0.2,rng = rng)

c = C@xx + 0.01
Q = 1e-7*diags_array(np.ones(n))
f = LogisticNLL(y,w)

In [26]:
from dataclasses import dataclass

@dataclass
class SolverSettings():
    max_precenter = 100
    max_iter = 200
    tol = 1e-7
    boundary_frac = 0.95
    gamma = 0.5
    min_mu = 1e-10
    tau_reg =1e-12
    init_merit_penalty =100.
    verbose = True


class GLMProblem():
    def __init__(
        self,
        f,A,Q,C,c,
        b=None,
        settings = None
        ):
        if settings is None:
            settings = SolverSettings()
        

        self.settings = settings
        
        m = A.shape[0]
        n = A.shape[1]
        assert A.shape[1] == C.shape[1]
        A = csc_array(A)

        #TODO:Separate solver dispatch for unconstrained
        assert C.shape[0] == len(c)
        C = csc_array(C)
        k = C.shape[0]
        self.k = k
        self.c = c

        if b is None:
            b = np.zeros(n)
        self.b = b
        
        
        if settings.verbose is True:
            print(f"{k} constraints")
            print(f"{n} variables")
            print(f"{m} rows in A")
        self.f = f
        self.A = A
        self.Q = Q
        self.C = C
        self.m = m
        self.n = n
        self.In = csc_array(eye_array(n))
        self.Ik = csc_array(eye_array(k))
    
    def initialize(self,x0=None,y0 = None,s0 = None):
        if x0 is None:
            x = np.zeros(self.n)
        else:
            x = np.copy(x0)
        
        if y0 is None:
            y = np.ones(self.k)
        else:
            y = np.copy(y0)
            assert np.min(y)>1e-8
        
        if s0 is None:
            s = np.maximum(self.c - self.C@x,1.)
        else:
            s = np.copy(s0)
        
        return x,y,s
    
    def KKT_res(self,x,g,y,s,mu):
        rx = g + self.C.T@y - self.b
        rp = self.C@x + s - self.c
        rc = y * s - mu
        return rx,rp,rc
    
    def solve_KKT(
        self,
        x,y,s,H,rx,rp,rc,mu,#mu,x unused for now
        solver = None):
        #Nesterov-Todd scaling
        # Quasi definite for inequality constraints, 
        # "normal equations" Hessian for GLM part
        w = np.sqrt(y/s)
        wC = self.C.multiply(w[:,None])
        #Including tiny tau-shift here
        #later may want separate matrix,
        #larger tau shift + iterative refine
        G = block_array(
            [
                [H+self.settings.tau_reg*self.In,wC.T],
                [wC,-self.Ik]
            ],format = 'csc'
        )
        rhs = np.hstack([-rx,-w*rp + (w/y) * rc])
        if solver is None:
            solver = qdldl.Solver(G)
        else:
            solver.update(G)
        sol = solver.solve(rhs)
        # linres = rhs - G@sol
        dx = sol[:self.n]
        dy = w*sol[self.n:]
        ds = -rp - self.C@dx
        return dx,ds,dy,solver
    
    def get_H(self,z):
        D = self.f.d2f(z)[:,None]
        AtDa = dot_product_mkl(self.A.T,csc_array(self.A.multiply(D)))
        return AtDa + self.Q
    
    def solve(
        self,
        x0=None,
        y0=None,
        s0=None
        ):
        x,y,s = self.initialize(x0,y0,s0)
        logger = PrettyLogger()
        settings = self.settings
        merit_penalty = settings.init_merit_penalty
        start = time.time()

        armijo_rat = 0.01
        max_beta_mul = 100
        max_linesearch = 50
        line_search_frac = 0.8
        beta = 1.

        
        def merit(x,y,s,mu,beta):
            fval = self.f(self.A@x)
            reg = (1/2) * x.T@self.Q@x
            barrier = -mu*np.sum(np.log(s))
            cons_viol = norm2(C@x +s - c)
            return fval + reg + barrier + beta * cons_viol,fval,cons_viol
    
        mu = 100.

        z = self.A@x
        H = self.get_H(z)
        gradf = self.A.T@self.f.d1f(z) + self.Q@x
        rx,rp,rc = self.KKT_res(x,gradf,y,s,mu)
        kkt_res = np.max(
                np.abs(np.hstack([rx,rp,rc+ mu]))#broadcasted (+mu), get r
            )
            

        
        solver = None    
        for i in range(settings.max_iter):
            if kkt_res<=settings.tol:
                break
            dx,ds,dy,solver = self.solve_KKT(x,y,s,H,rx,rp,rc,mu,solver)
            tmax = get_step_size(s,ds,y,dy,frac = settings.boundary_frac)
            t = tmax
            #_______________________________________
            x_gf = gradf - self.b
            s_gf = -mu/s

            x_gpen = C.T@rp
            s_gpen = rp

            fdec = np.dot(x_gf, dx) + np.dot(s_gf,ds)
            pen_dec = np.dot(x_gpen,dx) + np.dot(s_gpen,ds)
            if fdec + beta * pen_dec > 0:
                #Force descent direction on merit
                beta = -2  * fdec/pen_dec
            dz = A@dx
            t = tmax

            oldf = f(z) + (1/2) * np.dot(x,Q@x) - np.dot(self.b,x) - mu * np.sum(np.log(s))
            old_penval = norm2(rp)

            def eval_line(t):
                xstep = x + t* dx
                sstep = s + t* ds
                primal = f(z + t * dz) + (1/2) * (xstep).T@Q@(xstep) - np.dot(self.b,xstep)
                barrier =  - mu * np.sum(np.log(sstep))
                penalty = norm2(C@(xstep) + sstep - c)
                return primal,barrier,penalty

            primal,barrier,new_penval = eval_line(t)
            newf = primal + barrier

            #if doesn't satisfy armijo on merit fun
            if newf + beta * new_penval > oldf + beta * old_penval + t * armijo_rat * (fdec + beta * pen_dec):
                accepted = False
                #Try to increase penalty to accept step if we're infeasible
                if maxnorm(rp)>1e-10:
                    #Check if we can make it satisfy by increasing beta
                    beta_new = -((newf - oldf) - armijo_rat*t * (fdec + beta * pen_dec))/(new_penval - old_penval)
                    if beta_new <= max_beta_mul * beta:
                        beta = beta_new
                        accepted = True
                    else:
                        beta = max_beta_mul * beta
                if accepted ==False:
                    #proceed to a linesearch
                    for ls in range(max_linesearch):
                        t = line_search_frac * t
                        primal,barrier,new_penval = eval_line(t)
                        newf = primal + barrier

                        if newf + beta * new_penval < oldf + beta * old_penval + t * armijo_rat * (fdec + beta * pen_dec):
                            accepted = True
                            break

                if accepted ==False:
                    warn("Warning: Line search failed, giving current solution")
                    return x,logger.to_dataframe()
                    
            #____________________________________________________
            x = x + t*dx
            s = s + t*ds
            y = y + t*dy
            z = self.A@x
            H = self.get_H(z)
            gradf = self.A.T@self.f.d1f(z) + self.Q@x

            if kkt_res<=100 * mu:
                mu_est = np.dot(s,y)/self.k
                xi = np.min(s*y)/mu_est
                mu_lower = np.maximum(mu*0.01,settings.min_mu)
                mu = np.maximum(
                    mu_lower,
                    settings.gamma * 
                    np.minimum(
                    (1-settings.boundary_frac)*(1-xi)/xi + 0.1,2)**3 * mu_est
                    )
                        
            rx,new_rp,rc = self.KKT_res(x,gradf,y,s,mu)
            if maxnorm(new_rp)>1e-8 and np.linalg.norm(new_rp)>0.95*np.linalg.norm(rp):
                merit_penalty = 20*merit_penalty

            rp = new_rp
            kkt_res = np.max(
                np.abs(np.hstack([rx,rp,rc+ mu]))#broadcasted (+mu), get r
            )
            elapsed = time.time() - start
            logger.log(
                iter=i+1,
                primal = primal,
                cons_viol = maxnorm(rp),
                mu=mu,
                merit_pen=merit_penalty,
                Δx = maxnorm(t*dx),
                step=t,
                KKT_res=kkt_res,
                cum_time=elapsed,
            )
        return x,logger.to_dataframe()

In [27]:
n = 5
A = csc_array(diags_array(np.ones(n)))
C = -1*csc_array(diags_array(np.ones(n)))

Q = 1e-9 *csc_array(diags_array(np.ones(n)))
b = -2*np.ones(n)

w = 100000.*np.ones(n)
y = 0.*np.ones(n)

c = -1000*np.ones(n)
f = LogisticNLL(y,w)
problem = GLMProblem(f,A,Q,C,c,b = b)


problem.solve()

5 constraints
5 variables
5 rows in A
────────────────────────────────────────────────────────────────────────────────────────────────────────
│ iter │   primal   │ cons_viol  │    mu    │ merit_pen │    Δx     │   step    │  KKT_res  │ cum_time │
────────────────────────────────────────────────────────────────────────────────────────────────────────
│    1 │ 3.4611e+05 │ 1.0001e+03 │  1.0e+02 │  2.00e+03 │  1.85e-03 │ 9.472e-04 │  5.00e+04 │     0.01s │
│    2 │ 3.4611e+05 │ 1.0000e+03 │  1.0e+02 │  4.00e+04 │  1.34e-05 │ 4.748e-05 │  5.00e+04 │     0.02s │
│    3 │ 3.4615e+05 │ 1.0000e+03 │  1.0e+02 │  8.00e+05 │  1.54e-04 │ 2.529e-06 │  5.00e+04 │     0.03s │
│    4 │ 3.4623e+05 │ 1.0000e+03 │  1.0e+02 │  1.60e+07 │  3.19e-04 │ 4.381e-07 │  5.00e+04 │     0.04s │
│    5 │ 3.4665e+05 │ 1.0000e+03 │  1.0e+02 │  3.20e+08 │  1.70e-03 │ 1.703e-06 │  5.00e+04 │     0.04s │
│    6 │ 5.0080e+08 │ 0.0000e+00 │  1.0e+02 │  3.20e+08 │  1.00e+03 │ 1.000e+00 │  3.95e+07 │     0.05s │
│    7 │ 5.

/tmp/ipykernel_3153518/4068679918.py:175: RuntimeWarning: divide by zero encountered in scalar divide
  beta = -2  * fdec/pen_dec
/tmp/ipykernel_3153518/4068679918.py:194: RuntimeWarning: invalid value encountered in scalar multiply
  if newf + beta * new_penval > oldf + beta * old_penval + t * armijo_rat * (fdec + beta * pen_dec):
/tmp/ipykernel_3153518/4068679918.py:173: RuntimeWarning: invalid value encountered in scalar multiply
  if fdec + beta * pen_dec > 0:


(array([1000., 1000., 1000., 1000., 1000.]),
     iter        primal     cons_viol            mu    merit_pen            Δx  \
 0      1  3.461106e+05  1.000052e+03  1.000000e+02       2000.0  1.852687e-03   
 1      2  3.461073e+05  1.000004e+03  1.000000e+02      40000.0  1.342197e-05   
 2      3  3.461457e+05  1.000002e+03  1.000000e+02     800000.0  1.540417e-04   
 3      4  3.462255e+05  1.000001e+03  1.000000e+02   16000000.0  3.193320e-04   
 4      5  3.466496e+05  9.999997e+02  1.000000e+02  320000000.0  1.696616e-03   
 5      6  5.007969e+08  0.000000e+00  1.000000e+02  320000000.0  1.001573e+03   
 6      7  5.027045e+08  0.000000e+00  1.000000e+02  320000000.0  3.815207e+00   
 7      8  5.024545e+08  0.000000e+00  1.000000e+02  320000000.0  5.000000e-01   
 8      9  5.001322e+08  0.000000e+00  1.000000e+02  320000000.0  4.644440e+00   
 9     10  5.000161e+08  0.000000e+00  1.000000e+02  320000000.0  2.322220e-01   
 10    11  5.000105e+08  0.000000e+00  1.000000e+00  

In [30]:
from scipy.stats import Uniform


m = 10000
n = 500
samp = Uniform(a = -0.5,b = 0.5).sample


A = random_array((m,n),density = 0.01,data_sampler = lambda size:samp(size))
x_true = np.random.uniform(-0.1,1,n)

z_true = A@x_true

w = 100*np.ones(m)
y = np.random.binomial(w.astype(int),expit(z_true))/w

Q = 1.*diags_array(np.ones(n))

rng = np.random.default_rng(10)

xx = rng.normal(size = n)
samp = Uniform(a = -0.5,b = 0.5).sample

C = random_array((200,n),
                 density = 0.2,rng = rng)

c = C@xx + 0.01
Q = 1e-7*diags_array(np.ones(n))
f = LogisticNLL(y,w)
problem = GLMProblem(f,A,Q,C,c)


problem.solve()

200 constraints
500 variables
10000 rows in A
────────────────────────────────────────────────────────────────────────────────────────────────────────
│ iter │   primal   │ cons_viol  │    mu    │ merit_pen │    Δx     │   step    │  KKT_res  │ cum_time │
────────────────────────────────────────────────────────────────────────────────────────────────────────
│    1 │ 8.4154e+05 │ 2.6509e+00 │  4.7e+03 │  1.00e+02 │  3.50e+00 │ 8.488e-01 │  1.84e+03 │     0.06s │
│    2 │ 1.4451e+06 │ 2.3448e-13 │  4.7e+01 │  1.00e+02 │  7.07e+00 │ 1.000e+00 │  1.36e+04 │     0.11s │
│    3 │ 8.7601e+05 │ 2.4869e-13 │  4.7e+01 │  1.00e+02 │  5.52e+00 │ 1.000e+00 │  3.34e+03 │     0.16s │
│    4 │ 7.4821e+05 │ 1.0303e-13 │  3.1e+00 │  1.00e+02 │  3.40e+00 │ 1.000e+00 │  8.59e+02 │     0.21s │
│    5 │ 7.1161e+05 │ 3.9080e-14 │  3.1e+00 │  1.00e+02 │  7.58e-01 │ 9.336e-01 │  2.41e+02 │     0.25s │
│    6 │ 7.0887e+05 │ 3.7303e-14 │  3.6e+02 │  1.00e+02 │  9.56e-02 │ 2.127e-01 │  1.93e+02 │     0.30s │
│  

(array([ 1.75067281e-02, -1.20370864e-01, -4.29518711e-01, -5.82696235e-01,
        -3.17742276e-01,  1.05941507e-01,  8.99500792e-02, -5.63777651e-01,
        -3.65150317e-02, -4.18456703e-01, -4.28537287e-01, -4.82144041e-01,
        -3.52541033e-01, -4.98279252e-01, -1.85809839e-01, -2.60484752e-01,
         7.11652575e-03, -5.42203467e-01, -4.72175549e-02,  3.37646765e-01,
         3.47888425e-02, -1.91994281e-01, -1.43357986e-01, -1.94929909e-01,
         6.63301210e-02, -4.66435678e-01,  4.82155046e-02, -3.00183504e-01,
         6.38454945e-01, -6.21484472e-01, -5.82346861e-01,  3.06709938e-01,
         3.64985102e-01,  3.85449375e-01, -2.75463533e-01, -4.70797108e-01,
        -4.71934047e-01, -3.18395666e-01, -5.02748434e-01, -2.65568577e-01,
        -5.88315187e-01,  7.91966094e-02, -2.44498548e-01, -2.37187338e-01,
        -1.14345581e-01, -1.58866456e-01, -4.02871790e-01, -6.76532556e-01,
         3.74948068e-01,  2.30018996e-01, -2.98844494e-01,  1.36491534e-01,
        -3.2

In [ ]:
import cvxpy as cp
n = A.shape[1]

beta = cp.Variable(n)
lambd = cp.Parameter(nonneg=True)
neg_log_likelihood = cp.sum(
    cp.multiply(w,cp.logistic(A @ beta)- cp.multiply(y, A @ beta))
)
regularization = beta.T@Q@beta/2

problem  = cp.Problem(cp.Minimize(
    (neg_log_likelihood + regularization)),[C @ beta <= c]
)
problem.solve(verbose = True)

                                     CVXPY                                     
                                     v1.6.5                                    
(CVXPY) Apr 17 08:54:43 PM: Your problem has 500 variables, 200 constraints, and 0 parameters.
(CVXPY) Apr 17 08:54:43 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Apr 17 08:54:43 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Apr 17 08:54:43 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Apr 17 08:54:43 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Apr 17 08:54:43 PM: Compiling problem (target solver=CLARABEL

In [138]:
print(f(A@x) + (1/2) *x.T@Q@x)

cvx_x = beta.value
print(f(A@cvx_x) + (1/2) *cvx_x.T@Q@cvx_x)

68542.6735196966
68542.67355710643


In [22]:
cvx_x - x

array([ 1.88304223e-06,  6.61183275e-06,  1.96638301e-06, -4.12074586e-08,
       -6.02499955e-06,  1.10058918e-06, -6.10930961e-07, -1.26946839e-06,
        5.88587145e-07,  6.00444789e-07, -1.00792179e-08,  2.13289630e-06,
       -1.92164394e-06,  1.98446058e-06, -4.71918464e-06,  2.48029504e-06,
       -8.10489266e-07,  7.49122216e-06,  2.10166768e-06, -2.29564910e-06,
        1.44871601e-07, -2.89828361e-06, -5.15847388e-06,  9.14257737e-07,
       -1.74684648e-06,  1.02833231e-06, -1.56754350e-05, -5.31380687e-07,
        5.18384500e-07, -3.40708443e-07,  4.43635682e-06,  1.30159259e-06,
       -3.78598153e-06,  2.37993245e-07,  5.00879387e-07,  5.31418336e-07,
       -1.72539662e-06, -1.81282658e-06, -2.21485332e-06, -2.74029104e-06,
       -3.99108030e-06,  1.21493751e-06,  6.06727824e-08,  1.36522697e-06,
        5.72811232e-07,  2.98111301e-06,  1.10734558e-07, -3.51312001e-06,
       -2.34090067e-06, -7.99061721e-07, -1.09719711e-06,  4.94875204e-07,
       -2.65096531e-06,  

In [177]:
print(problem.value -f(A@x) - (1/2) *x.T@Q@x)

-6.8239253430135705e-06
